Rosa Garza, July 5, 2018 (Day 4)


In [2]:
import numpy as np
import matplotlib as mpl
import pandas as pd
import pylab as plt
import csv
import os
import sys
import matplotlib.pyplot as plt
#os.mkdir("plots")

In order to get help on opening and writing to a new csv file, I used the following link:
https://stackoverflow.com/questions/30938840/python-csv-read-file-and-select-columns-and-write-to-new-csv-file

as well as the link below: 

http://www.pythonforbeginners.com/systems-programming/using-the-csv-module-in-python/

In [3]:
# make sure to have the gene data within your current working directory
gene_dat=pd.read_csv('vcf_FA.csv',index_col='Unnamed: 0',encoding='utf-8')
matrix=gene_dat.as_matrix()
#print(matrix.shape[0])
#matrix[0]

In [4]:
gene_mut=np.zeros((matrix.shape[0],int(np.ceil((matrix.shape[1])/2))))
#print(gene_mut.shape)

In [5]:
# will go through each row (# of observations)
for rows in range(matrix.shape[0]):
    for cols in range(0,matrix.shape[1]-1,2):
        # within those observations, it will go until the 2nd to last column (excludes phenotype)
        #sanity check (next 2 lines)
        #print('cols val is:',cols)
        #print('matrix values are: ',matrix[rows][cols],matrix[rows][cols+1])
        #checks to see if either the L or R gene has a mutation
        if(matrix[rows][cols]==1 or matrix[rows][cols+1]==1):
            #if it does, then in the copied matrix, a 1 will be input
            gene_mut[rows][int(cols/2)]=1
            #reason for int(cols/2) is to match up indexes with the pairing # that it's on
            #ex: 0&1 gene output val. will be in the 0 index of copied matrix
            # 2&3 gene output val. will be in the 1 index, 4*6 in the 2nd index
        #sanity check of what is being output
        #print('copy matrix val is:',gene_mut[rows][int(cols/2)], 'for row:',rows,'for col:',int(cols/2))

The line below I created to get the last column of the main matrix and put those values into the last column of the new copied dataset being created.

In [6]:
gene_mut[:,-1]=matrix[:,-1] # gets last column from original matrix, puts it as last column for copied matrix

Below was just a sanity check to make sure my code was performing what was needed.

In [6]:
#gene_mut

array([[   1.,    0.,    1., ...,    1.,    0.,  196.],
       [   1.,    1.,    1., ...,    0.,    0.,  173.],
       [   1.,    0.,    1., ...,    0.,    1.,  198.],
       ..., 
       [   1.,    0.,    1., ...,    0.,    0.,  168.],
       [   1.,    0.,    1., ...,    0.,    0.,  155.],
       [   1.,    0.,    1., ...,    0.,    1.,  145.]])

In [7]:
# getting column names of genes without the L or R
names=gene_dat.columns
new_colnames=[]
for n in range(0,len(names),2):
    new_colnames.append(names[n].split("-")[0])
#print(new_colnames)

For the code below, this helped me,
https://stackoverflow.com/questions/11106536/adding-row-column-headers-to-numpy-arrays

In [19]:
copied_data=pd.DataFrame(gene_mut,index=gene_dat.index,columns=new_colnames)

In [20]:
copied_data.to_csv("copiedGeneData.csv")

In [21]:
rdm13=pd.read_csv('copiedGeneData.csv',index_col='Unnamed: 0',encoding='utf-8')
rdm13=rdm13.as_matrix()


In [22]:
def num_gene_mutations(raw_data):
    n_ones=[]
    # 0 to number of total rows
    for row in range(0,(raw_data.shape[0])):
        #calculating each person's number of gene mutations, excluding last column with phenotype levels
        n_ones.append(sum(rdm13[row,:-1]))
    return n_ones

In [23]:
sum_gene_mut=num_gene_mutations(rdm13)
#print(sum_gene_mut)
#for printing total number of people based on number of gene mutations
plt.hist(sum_gene_mut,bins=13)
plt.xlabel("Total Gene Mutations")
plt.ylabel("Total People")
plt.title("Histogram of Gene Mutations Based on Reduced Data Set")
plt.savefig("plots/SumMutationInfo.png")
#plt.show()


In [24]:
# getting an an array of the opposites. We were looking at the number of 1's within each row (num of mutations
#each person had), then we decided to have a matrix which calcualted num of 0's
nzeros=13-np.asarray(sum_gene_mut)
#print(nzeros)

In [25]:
# returning matrix of people with 0's that are less than floor(13/2)
def minority_0matrix(RDM,n_ones):
    col=RDM.shape[1]
    zero_mat=[]
    #basically 0- number of people
    for i in range (0,len(n_ones)):
        #if the number of 0's is less than or equal to floor(13/2), meaning more 0s than 1s
        if((RDM.shape[1]-1)-n_ones[i]<=np.floor((col-1)/2)):
            #we will add that row to a list
            zero_mat.append(RDM[i,])
    #getting the list and reshaping it to be of #ppl by #columns
    return np.array(zero_mat).reshape(len(zero_mat),col)

In [26]:
minority_0matrix(rdm13,nzeros).shape
# returns dimension of matrix, meaning there's a matrix 21 x 14 in which these people have 7+ gene mutations
# we will be flipping these matrices in the future
#rdm13.shape[1]

(21, 14)

This was the code implemented in order to have Lillian calculate the f_i's for people of 5 mutations.

In [27]:
# matrix in which people have 5 gene mutations in total
rdm13[np.asarray(sum_gene_mut)==5]

array([[   1.,    0.,    1.,    1.,    1.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    1.,  198.],
       [   1.,    0.,    0.,    1.,    1.,    0.,    1.,    1.,    0.,
           0.,    0.,    0.,    0.,  176.],
       [   1.,    0.,    0.,    0.,    1.,    1.,    0.,    1.,    0.,
           1.,    0.,    0.,    0.,  164.],
       [   1.,    0.,    1.,    1.,    0.,    1.,    1.,    0.,    0.,
           0.,    0.,    0.,    0.,  149.],
       [   1.,    0.,    1.,    1.,    0.,    1.,    1.,    0.,    0.,
           0.,    0.,    0.,    0.,  155.]])

https://stackoverflow.com/questions/16348953/creating-a-folder-and-set-as-working-directory-for-saving-stdout-files-and-image

In [29]:
import sys
import os
#os.mkdir('adj_matrices')
filename = open(os.path.join('adj_matices','filename.txt'),'w')
#sys.stdout = filename


FileNotFoundError: [Errno 2] No such file or directory: 'adj_matices\\filename.txt'

In [22]:
# Goal for Sunday: make plot of data which goes from # small gene mutations, to greatest number of gene mutations
# plot their hemoglobin levels

https://stackoverflow.com/questions/15726618/array-indexing-in-python

In [30]:
tuple_sums=[]
for index,element in enumerate(sum_gene_mut):
    tuple_sums.append((index,element))
#print(tuple_sums)

In [112]:
#tuple_sums[1][0]

1

https://stackoverflow.com/questions/10695139/sort-a-list-of-tuples-by-2nd-item-integer-value

In [31]:
#print(sorted(tuple_sums,key=lambda x: x[1]))

In [32]:
tuple_sums_sorted=sorted(tuple_sums,key=lambda x: x[1])
#tuple_sums_sorted

In [114]:
#gene_mut[tuple_sums_sorted[0][0]][-1]

198.0

In [33]:
list_5=[]
list_6=[]
list_7=[]
list_8=[]
list_9=[]
list_10=[]
for i in range(len(tuple_sums_sorted)):
    if (tuple_sums[i][1]==5):
        list_5.append(tuple_sums_sorted[i])
    if (tuple_sums[i][1]==6):
        list_6.append(tuple_sums_sorted[i])
    if (tuple_sums[i][1]==7):
        list_7.append(tuple_sums_sorted[i])
    if (tuple_sums[i][1]==8):
        list_8.append(tuple_sums_sorted[i])
    if (tuple_sums[i][1]==9):
        list_9.append(tuple_sums_sorted[i])
    if (tuple_sums[i][1]==10):
        list_10.append(tuple_sums_sorted[i])
    #print(gene_mut[tuple_sums_sorted[i][0]][-1])

In [36]:
list_5_hemo=[]
for i in range(len(list_5)):
    list_5_hemo.append(gene_mut[list_5[i][0]][-1])
    #print(gene_mut[list_5[i][0]][-1])

    
plt.scatter(x=range(len(list_5_hemo)),y=list_5_hemo)
plt.xlabel("Person (Ordering based on appearance in csv)")
plt.ylabel("Hemoglobin Level")
plt.title("Scatter Plot of People with 5 gene mutations")
plt.savefig("plots/HemoLevels_5GM.png")
#plt.show()

In [37]:
list_6_hemo=[]
for i in range(len(list_6)):
    list_6_hemo.append(gene_mut[list_6[i][0]][-1])
    #print(gene_mut[list_6[i][0]][-1])

    
plt.scatter(x=range(len(list_6_hemo)),y=list_6_hemo)
plt.xlabel("Person (Ordering based on appearance in csv)")
plt.ylabel("Hemoglobin Level")
plt.title("Scatter Plot of People with 6 gene mutations")
plt.savefig("plots/HemoLevels_6GM.png")
#plt.show()

In [38]:
list_7_hemo=[]
for i in range(len(list_7)):
    list_7_hemo.append(gene_mut[list_7[i][0]][-1])
    #print(gene_mut[list_7[i][0]][-1])

    
plt.scatter(x=range(len(list_7_hemo)),y=list_7_hemo)
plt.xlabel("Person (Ordering based on appearance in csv)")
plt.ylabel("Hemoglobin Level")
plt.title("Scatter Plot of People with 7 gene mutations")
plt.savefig("plots/HemoLevels_7GM.png")
#plt.show()

In [39]:
list_8_hemo=[]
for i in range(len(list_8)):
    list_8_hemo.append(gene_mut[list_8[i][0]][-1])
    #print(gene_mut[list_8[i][0]][-1])

    
plt.scatter(x=range(len(list_8_hemo)),y=list_8_hemo)
plt.xlabel("Person (Ordering based on appearance in csv)")
plt.ylabel("Hemoglobin Level")
plt.title("Scatter Plot of People with 8 gene mutations")
plt.savefig("plots/HemoLevels_8GM.png")
#plt.show()

In [40]:
list_9_hemo=[]
for i in range(len(list_9)):
    list_9_hemo.append(gene_mut[list_9[i][0]][-1])
    #print(gene_mut[list_9[i][0]][-1])

    
plt.scatter(x=range(len(list_9_hemo)),y=list_9_hemo)
plt.xlabel("Person (Ordering based on appearance in csv)")
plt.ylabel("Hemoglobin Level")
plt.title("Scatter Plot of People with 9 gene mutations")
plt.savefig("plots/HemoLevels_9GM.png")
#plt.show()

In [41]:
list_10_hemo=[]
for i in range(len(list_10)):
    list_10_hemo.append(gene_mut[list_10[i][0]][-1])
    #print(gene_mut[list_10[i][0]][-1])
plt.scatter(x=range(len(list_10_hemo)),y=list_10_hemo)
plt.xlabel("Person (Ordering based on appearance in csv)")
plt.ylabel("Hemoglobin Level")
plt.title("Scatter Plot of People with 10 gene mutations")
plt.savefig("plots/HemoLevels_10GM.png")
#plt.show()

For code below:
https://stackoverflow.com/questions/4800811/accessing-a-value-in-a-tuple-that-is-in-a-list

In [42]:
unique_sum_mut=np.unique([x[1] for x in tuple_sums])

In [43]:
plt.scatter(x=range(len(list_5_hemo)),y=list_5_hemo,c='red',label="5")
plt.scatter(x=range(len(list_6_hemo)),y=list_6_hemo,c='green',label="6")
plt.scatter(x=range(len(list_7_hemo)),y=list_7_hemo,c='blue',label="7")
plt.scatter(x=range(len(list_8_hemo)),y=list_8_hemo,c='orange',label="8")
plt.scatter(x=range(len(list_9_hemo)),y=list_9_hemo,c='cyan',label="9")
plt.scatter(x=range(len(list_10_hemo)),y=list_10_hemo,c='magenta',label="10")
plt.legend(loc="best",bbox_to_anchor=(1,1),title="# Gene Mutations")
plt.ylabel("Hemoglobin Level")
plt.title("Hemoglobin Level Based on Num. Gene Mutations")
plt.savefig("plots/HemoLevels_All.png",bbox_inches="tight")
plt.tight_layout()
#plt.show()

For help with plotting and colors:
https://stackoverflow.com/questions/14885895/color-by-column-values-in-matplotlib

In [45]:
# CODE USED AS GUIDANCE
#plt.scatter(x=range(len(tuple_sums)), waterUsage['water_amount'],\
#            c=waterUsage['third_column'], cmap=plt.cm.autumn)

https://pythonspot.com/matplotlib-bar-chart/

In [46]:
first_order_inter=np.asarray([ 518.15384615, -323.84615385,  178.15384615,  354.15384615,
         214.15384615,  144.15384615,  156.15384615,   16.15384615,
        -323.84615385, -159.84615385, -323.84615385, -323.84615385,
        -125.84615385])

In [51]:
first_order_inter=first_order_inter.round(decimals=3)
#first_order_inter

In [50]:
y_pos = np.arange(len(first_order_inter))
#print(min(first_order_inter))

plt.bar(y_pos,first_order_inter, align='center')
plt.title('1st order effects within groupings of 5 mutations')
plt.ylim((min(first_order_inter)-100),(max(first_order_inter)+100))
plt.savefig("plots/FirstOrder_13_5.png")
#plt.show()

In [52]:
sec_order_inter=np.asarray([-178.60606061,  140.84848485,  252.84848485,  163.75757576,
         119.21212121,  126.84848485,   37.75757576, -178.60606061,
         -74.24242424, -178.60606061, -178.60606061,  -52.60606061,
         -54.96969697, -118.96969697,  -68.06060606,  -42.60606061,
         -46.96969697,    3.93939394,  127.57575758,   67.93939394,
         127.57575758,  127.57575758,   55.57575758,  200.48484848,
         -52.60606061,   78.84848485,   74.48484848, -178.60606061,
         -54.96969697, -114.60606061,  -54.96969697,  -54.96969697,
          71.03030303,   59.39393939,   14.84848485,  186.48484848,
         -66.60606061, -118.96969697, -178.60606061, -118.96969697,
        -118.96969697,    7.03030303,  -74.24242424,  -66.60606061,
         148.3030303 ,  -68.06060606,   36.3030303 ,  -68.06060606,
         -68.06060606,   57.93939394,   86.84848485,   -2.24242424,
         -42.60606061,   61.75757576,  -42.60606061,  -42.60606061,
        -114.60606061,    5.39393939,  -46.96969697, -106.60606061,
         -46.96969697,  -46.96969697, -118.96969697,    3.93939394,
         108.3030303 ,    3.93939394,    3.93939394,  -68.06060606,
          67.93939394,  127.57575758,  127.57575758,   55.57575758,
          67.93939394,   67.93939394,   -4.06060606,  127.57575758,
          55.57575758,   55.57575758])

In [53]:
y_pos_2 = np.arange(len(sec_order_inter))
print(min(sec_order_inter))

plt.bar(y_pos_2,sec_order_inter, align='center')
plt.title('2nd order effects within groupings of 5 mutations')
plt.ylim((min(sec_order_inter)-100),(max(sec_order_inter)+100))
plt.savefig("plots/SecOrder_13_5.png")
#plt.show()

-178.60606061


In [54]:
third_order_inter=np.asarray([ -39.16363636,  -74.36363636,  -46.36363636,  -32.36363636,
         -34.76363636,   -6.76363636,   61.23636364,   28.43636364,
          61.23636364,   61.23636364,   21.63636364,  159.9030303 ,
         -14.76363636,   69.9030303 ,   67.5030303 , -107.16363636,
         -39.16363636,  -71.96363636,  -39.16363636,  -39.16363636,
          53.23636364,   67.36969697,   34.7030303 ,  149.63636364,
         -25.03030303,  -74.36363636, -107.16363636,  -74.36363636,
         -74.36363636,   18.03636364,  -30.63030303,  -25.03030303,
         112.3030303 ,  -46.36363636,   30.16969697,  -46.36363636,
         -46.36363636,   46.03636364,   74.3030303 ,    8.96969697,
         -32.36363636,   44.16969697,  -32.36363636,  -32.36363636,
         -71.96363636,   14.56969697,  -34.76363636,  -67.56363636,
         -34.76363636,  -34.76363636,  -74.36363636,   -6.76363636,
          69.76969697,   -6.76363636,   -6.76363636,  -46.36363636,
          28.43636364,   61.23636364,   61.23636364,   21.63636364,
          28.43636364,   28.43636364,  -11.16363636,   61.23636364,
          21.63636364,   21.63636364,  -61.03030303,   21.63636364,
         -23.03030303,  -21.43030303,   61.23636364,   15.9030303 ,
          37.76969697,   15.9030303 ,   15.9030303 ,  -23.6969697 ,
         -13.56363636,    0.43636364,  -56.63030303,   26.03636364,
          39.36969697,   61.23636364,   39.36969697,   39.36969697,
          -0.23030303,   28.43636364,   26.03636364,  -47.2969697 ,
          20.7030303 ,  -12.0969697 ,   20.7030303 ,   20.7030303 ,
         -18.8969697 ,  -25.96363636,    2.03636364,   11.36969697,
         -21.43030303,   11.36969697,   11.36969697,   37.76969697,
          -0.36363636,   12.96969697,   34.83636364,   12.96969697,
          12.96969697,   39.36969697,   -5.6969697 ,  -38.4969697 ,
          -5.6969697 ,   -5.6969697 ,   20.7030303 ,  -29.16363636,
         -51.03030303,  -51.03030303,  -24.63030303,  -29.16363636,
         -29.16363636,   -2.76363636,  -51.03030303,  -24.63030303,
         -24.63030303,   18.03636364,  102.7030303 ,   45.63636364,
         -74.36363636,  -61.03030303,  -39.16363636,  -61.03030303,
         -61.03030303,   31.36969697,  -71.96363636,  -74.36363636,
         -46.36363636,   21.63636364,  -11.16363636,   21.63636364,
          21.63636364,  114.03636364,  142.3030303 ,  -32.36363636,
         -23.03030303,  -55.83030303,  -23.03030303,  -23.03030303,
         -62.63030303,  -34.76363636,  -21.43030303,    0.43636364,
         -21.43030303,  -21.43030303,  -61.03030303,   61.23636364,
          28.43636364,   61.23636364,   61.23636364,   21.63636364,
          37.76969697,   15.9030303 ,   15.9030303 ,  -23.6969697 ,
          37.76969697,   37.76969697,   -1.83030303,   15.9030303 ,
         -23.6969697 ,  -23.6969697 , -107.16363636,    7.76969697,
          35.76969697,  -13.56363636,  -46.36363636,  -13.56363636,
         -13.56363636,   78.83636364,  107.1030303 ,  -67.56363636,
           0.43636364,  -32.36363636,    0.43636364,    0.43636364,
         -39.16363636,   47.36969697,  -56.63030303,  -34.76363636,
         -56.63030303,  -56.63030303,  -96.23030303,   26.03636364,
          -6.76363636,   26.03636364,   26.03636364,  -13.56363636,
          61.23636364,   39.36969697,   39.36969697,   -0.23030303,
          61.23636364,   61.23636364,   21.63636364,   39.36969697,
          -0.23030303,   -0.23030303,  -67.56363636,   69.76969697,
          28.43636364,  104.96969697,   28.43636364,   28.43636364,
         -11.16363636,   75.36969697,   26.03636364,   -6.76363636,
          26.03636364,   26.03636364,  -13.56363636,  -47.2969697 ,
          29.23636364,  -47.2969697 ,  -47.2969697 ,  -86.8969697 ,
         -12.0969697 ,   20.7030303 ,   20.7030303 ,  -18.8969697 ,
         -12.0969697 ,  -12.0969697 ,  -51.6969697 ,   20.7030303 ,
         -18.8969697 ,  -18.8969697 ,  -93.96363636,  -25.96363636,
         -58.76363636,  -25.96363636,  -25.96363636,    0.43636364,
           2.03636364,   78.56969697,    2.03636364,    2.03636364,
          28.43636364,  -21.43030303,   11.36969697,   11.36969697,
          37.76969697,  -21.43030303,  -21.43030303,    4.96969697,
          11.36969697,   37.76969697,   37.76969697,   -0.36363636,
         -33.16363636,   -0.36363636,   -0.36363636,   26.03636364,
          34.83636364,   12.96969697,   12.96969697,   39.36969697,
          34.83636364,   34.83636364,   61.23636364,   12.96969697,
          39.36969697,   39.36969697,  -38.4969697 ,   -5.6969697 ,
          -5.6969697 ,   20.7030303 ,  -38.4969697 ,  -38.4969697 ,
         -12.0969697 ,   -5.6969697 ,   20.7030303 ,   20.7030303 ,
         -29.16363636,  -29.16363636,   -2.76363636,  -51.03030303,
         -24.63030303,  -24.63030303,  -29.16363636,   -2.76363636,
          -2.76363636,  -24.63030303])

In [55]:
y_pos_3 = np.arange(len(third_order_inter))
print(min(third_order_inter))

plt.bar(y_pos_3,third_order_inter, align='center')
plt.title('3rd order effects within groupings of 5 mutations')
plt.ylim((min(third_order_inter)-50),(max(third_order_inter)+50))
plt.savefig("plots/ThirdOrder_13_5.png")
#plt.show()

-107.16363636


In [56]:
fourth_order_inter=np.asarray([ -3.94523810e+01,   6.50000000e+00,  -1.65952381e+01,
         -1.58809524e+01,   3.00714286e+01,   9.83333333e+00,
          1.95952381e+01,   9.83333333e+00,   9.83333333e+00,
         -1.37380952e+01,  -1.44523810e+01,  -6.11904762e+00,
         -3.68333333e+01,   9.11904762e+00,   2.03095238e+01,
          3.00714286e+01,   2.03095238e+01,   2.03095238e+01,
         -3.26190476e+00,   1.05476190e+01,   9.11904762e+00,
         -2.85000000e+01,   1.19761905e+01,  -7.54761905e+00,
          1.19761905e+01,   1.19761905e+01,  -1.15952381e+01,
         -1.79047619e+01,  -1.23809524e+00,   7.80952381e+00,
         -1.17142857e+01,   7.80952381e+00,   7.80952381e+00,
          1.95952381e+01,  -2.66666667e+00,   8.52380952e+00,
          1.82857143e+01,   8.52380952e+00,   8.52380952e+00,
          2.03095238e+01,   1.90476190e-01,  -1.93333333e+01,
          1.90476190e-01,   1.90476190e-01,   1.19761905e+01,
         -1.02857143e+01,  -2.00476190e+01,  -2.00476190e+01,
         -8.26190476e+00,  -1.02857143e+01,  -1.02857143e+01,
          1.50000000e+00,  -2.00476190e+01,  -8.26190476e+00,
         -8.26190476e+00,   3.18571429e+01,   9.69761905e+01,
          6.62619048e+01,  -5.06428571e+01,  -3.94523810e+01,
         -2.96904762e+01,  -3.94523810e+01,  -3.94523810e+01,
          4.30476190e+01,  -4.92142857e+01,  -5.06428571e+01,
         -3.39761905e+01,   6.50000000e+00,  -1.30238095e+01,
          6.50000000e+00,   6.50000000e+00,   8.90000000e+01,
          1.20547619e+02,  -2.56428571e+01,  -1.65952381e+01,
         -3.61190476e+01,  -1.65952381e+01,  -1.65952381e+01,
         -4.01666667e+01,  -2.70714286e+01,  -1.58809524e+01,
         -6.11904762e+00,  -1.58809524e+01,  -1.58809524e+01,
         -3.94523810e+01,   3.00714286e+01,   1.05476190e+01,
          3.00714286e+01,   3.00714286e+01,   6.50000000e+00,
          1.95952381e+01,   9.83333333e+00,   9.83333333e+00,
         -1.37380952e+01,   1.95952381e+01,   1.95952381e+01,
         -3.97619048e+00,   9.83333333e+00,  -1.37380952e+01,
         -1.37380952e+01,  -7.01666667e+01,   2.26904762e+01,
          3.93571429e+01,  -1.44523810e+01,  -3.39761905e+01,
         -1.44523810e+01,  -1.44523810e+01,   6.80476190e+01,
          9.95952381e+01,  -4.65952381e+01,  -6.11904762e+00,
         -2.56428571e+01,  -6.11904762e+00,  -6.11904762e+00,
         -2.96904762e+01,   4.62619048e+01,  -3.68333333e+01,
         -2.70714286e+01,  -3.68333333e+01,  -3.68333333e+01,
         -6.04047619e+01,   9.11904762e+00,  -1.04047619e+01,
          9.11904762e+00,   9.11904762e+00,  -1.44523810e+01,
          3.00714286e+01,   2.03095238e+01,   2.03095238e+01,
         -3.26190476e+00,   3.00714286e+01,   3.00714286e+01,
          6.50000000e+00,   2.03095238e+01,  -3.26190476e+00,
         -3.26190476e+00,  -4.65952381e+01,   5.79285714e+01,
          1.05476190e+01,   7.88809524e+01,   1.05476190e+01,
          1.05476190e+01,  -1.30238095e+01,   6.29285714e+01,
          9.11904762e+00,  -1.04047619e+01,   9.11904762e+00,
          9.11904762e+00,  -1.44523810e+01,  -2.85000000e+01,
          3.98333333e+01,  -2.85000000e+01,  -2.85000000e+01,
         -5.20714286e+01,  -7.54761905e+00,   1.19761905e+01,
          1.19761905e+01,  -1.15952381e+01,  -7.54761905e+00,
         -7.54761905e+00,  -3.11190476e+01,   1.19761905e+01,
         -1.15952381e+01,  -1.15952381e+01,  -5.83809524e+01,
         -1.79047619e+01,  -3.74285714e+01,  -1.79047619e+01,
         -1.79047619e+01,  -6.11904762e+00,  -1.23809524e+00,
          6.70952381e+01,  -1.23809524e+00,  -1.23809524e+00,
          1.05476190e+01,  -1.17142857e+01,   7.80952381e+00,
          7.80952381e+00,   1.95952381e+01,  -1.17142857e+01,
         -1.17142857e+01,   7.14285714e-02,   7.80952381e+00,
          1.95952381e+01,   1.95952381e+01,  -2.66666667e+00,
         -2.21904762e+01,  -2.66666667e+00,  -2.66666667e+00,
          9.11904762e+00,   1.82857143e+01,   8.52380952e+00,
          8.52380952e+00,   2.03095238e+01,   1.82857143e+01,
          1.82857143e+01,   3.00714286e+01,   8.52380952e+00,
          2.03095238e+01,   2.03095238e+01,  -1.93333333e+01,
          1.90476190e-01,   1.90476190e-01,   1.19761905e+01,
         -1.93333333e+01,  -1.93333333e+01,  -7.54761905e+00,
          1.90476190e-01,   1.19761905e+01,   1.19761905e+01,
         -1.02857143e+01,  -1.02857143e+01,   1.50000000e+00,
         -2.00476190e+01,  -8.26190476e+00,  -8.26190476e+00,
         -1.02857143e+01,   1.50000000e+00,   1.50000000e+00,
         -8.26190476e+00,  -3.26190476e+00,  -2.63571429e+01,
         -8.07142857e+00,   2.03095238e+01,   1.76428571e+01,
          9.83333333e+00,   1.76428571e+01,   1.76428571e+01,
         -5.92857143e+00,   1.95952381e+01,   2.03095238e+01,
          1.19761905e+01,  -8.26190476e+00,   1.50000000e+00,
         -8.26190476e+00,  -8.26190476e+00,  -3.18333333e+01,
         -3.81428571e+01,   7.80952381e+00,   6.42857143e+00,
          1.61904762e+01,   6.42857143e+00,   6.42857143e+00,
          1.82142857e+01,   8.52380952e+00,   5.85714286e+00,
         -1.95238095e+00,   5.85714286e+00,   5.85714286e+00,
          1.76428571e+01,  -2.00476190e+01,  -1.02857143e+01,
         -2.00476190e+01,  -2.00476190e+01,  -8.26190476e+00,
         -1.16666667e+01,  -3.85714286e+00,  -3.85714286e+00,
          7.92857143e+00,  -1.16666667e+01,  -1.16666667e+01,
          1.19047619e-01,  -3.85714286e+00,   7.92857143e+00,
          7.92857143e+00,   3.00714286e+01,  -6.42857143e-01,
         -8.97619048e+00,   2.21428571e+00,   1.19761905e+01,
          2.21428571e+00,   2.21428571e+00,  -2.13571429e+01,
         -2.76666667e+01,   1.82857143e+01,  -1.95238095e+00,
          7.80952381e+00,  -1.95238095e+00,  -1.95238095e+00,
          9.83333333e+00,  -1.24285714e+01,   1.63333333e+01,
          8.52380952e+00,   1.63333333e+01,   1.63333333e+01,
          2.81190476e+01,  -9.57142857e+00,   1.90476190e-01,
         -9.57142857e+00,  -9.57142857e+00,   2.21428571e+00,
         -2.00476190e+01,  -1.22380952e+01,  -1.22380952e+01,
         -4.52380952e-01,  -2.00476190e+01,  -2.00476190e+01,
         -8.26190476e+00,  -1.22380952e+01,  -4.52380952e-01,
         -4.52380952e-01,   1.82857143e+01,  -1.93333333e+01,
         -1.02857143e+01,  -2.98095238e+01,  -1.02857143e+01,
         -1.02857143e+01,   1.50000000e+00,  -2.07619048e+01,
         -9.57142857e+00,   1.90476190e-01,  -9.57142857e+00,
         -9.57142857e+00,   2.21428571e+00,   1.46666667e+01,
         -4.85714286e+00,   1.46666667e+01,   1.46666667e+01,
          2.64523810e+01,   4.19047619e+00,  -5.57142857e+00,
         -5.57142857e+00,   6.21428571e+00,   4.19047619e+00,
          4.19047619e+00,   1.59761905e+01,  -5.57142857e+00,
          6.21428571e+00,   6.21428571e+00,   2.77142857e+01,
          7.47619048e+00,   1.72380952e+01,   7.47619048e+00,
          7.47619048e+00,  -1.95238095e+00,  -8.57142857e-01,
         -2.03809524e+01,  -8.57142857e-01,  -8.57142857e-01,
         -1.02857143e+01,   7.52380952e+00,  -2.23809524e+00,
         -2.23809524e+00,  -1.16666667e+01,   7.52380952e+00,
          7.52380952e+00,  -1.90476190e+00,  -2.23809524e+00,
         -1.16666667e+01,  -1.16666667e+01,  -1.42857143e-01,
          9.61904762e+00,  -1.42857143e-01,  -1.42857143e-01,
         -9.57142857e+00,  -1.06190476e+01,  -2.80952381e+00,
         -2.80952381e+00,  -1.22380952e+01,  -1.06190476e+01,
         -1.06190476e+01,  -2.00476190e+01,  -2.80952381e+00,
         -1.22380952e+01,  -1.22380952e+01,   1.36190476e+01,
          3.85714286e+00,   3.85714286e+00,  -5.57142857e+00,
          1.36190476e+01,   1.36190476e+01,   4.19047619e+00,
          3.85714286e+00,  -5.57142857e+00,  -5.57142857e+00,
          1.22380952e+01,   1.22380952e+01,   2.80952381e+00,
          2.00476190e+01,   1.06190476e+01,   1.06190476e+01,
          1.22380952e+01,   2.80952381e+00,   2.80952381e+00,
          1.06190476e+01,  -2.96904762e+01,  -6.04047619e+01,
         -1.44523810e+01,  -3.26190476e+00,   6.50000000e+00,
         -3.26190476e+00,  -3.26190476e+00,   7.92380952e+01,
          1.10785714e+02,  -6.11904762e+00,  -2.63571429e+01,
         -1.65952381e+01,  -2.63571429e+01,  -2.63571429e+01,
         -4.99285714e+01,  -3.68333333e+01,  -8.07142857e+00,
         -1.58809524e+01,  -8.07142857e+00,  -8.07142857e+00,
         -3.16428571e+01,   2.03095238e+01,   3.00714286e+01,
          2.03095238e+01,   2.03095238e+01,  -3.26190476e+00,
          9.83333333e+00,   1.76428571e+01,   1.76428571e+01,
         -5.92857143e+00,   9.83333333e+00,   9.83333333e+00,
         -1.37380952e+01,   1.76428571e+01,  -5.92857143e+00,
         -5.92857143e+00,  -6.11904762e+00,   1.05476190e+01,
          1.95952381e+01,   7.14285714e-02,   1.95952381e+01,
          1.95952381e+01,  -3.97619048e+00,   9.11904762e+00,
          2.03095238e+01,   3.00714286e+01,   2.03095238e+01,
          2.03095238e+01,  -3.26190476e+00,   1.19761905e+01,
         -7.54761905e+00,   1.19761905e+01,   1.19761905e+01,
         -1.15952381e+01,   1.50000000e+00,  -8.26190476e+00,
         -8.26190476e+00,  -3.18333333e+01,   1.50000000e+00,
          1.50000000e+00,  -2.20714286e+01,  -8.26190476e+00,
         -3.18333333e+01,  -3.18333333e+01,  -1.79047619e+01,
         -3.81428571e+01,  -2.83809524e+01,  -3.81428571e+01,
         -3.81428571e+01,  -2.63571429e+01,   7.80952381e+00,
         -1.17142857e+01,   7.80952381e+00,   7.80952381e+00,
          1.95952381e+01,   1.61904762e+01,   6.42857143e+00,
          6.42857143e+00,   1.82142857e+01,   1.61904762e+01,
          1.61904762e+01,   2.79761905e+01,   6.42857143e+00,
          1.82142857e+01,   1.82142857e+01,   8.52380952e+00,
          1.82857143e+01,   8.52380952e+00,   8.52380952e+00,
          2.03095238e+01,  -1.95238095e+00,   5.85714286e+00,
          5.85714286e+00,   1.76428571e+01,  -1.95238095e+00,
         -1.95238095e+00,   9.83333333e+00,   5.85714286e+00,
          1.76428571e+01,   1.76428571e+01,  -1.02857143e+01,
         -2.00476190e+01,  -2.00476190e+01,  -8.26190476e+00,
         -1.02857143e+01,  -1.02857143e+01,   1.50000000e+00,
         -2.00476190e+01,  -8.26190476e+00,  -8.26190476e+00,
         -1.16666667e+01,  -1.16666667e+01,   1.19047619e-01,
         -3.85714286e+00,   7.92857143e+00,   7.92857143e+00,
         -1.16666667e+01,   1.19047619e-01,   1.19047619e-01,
          7.92857143e+00,  -2.70714286e+01,  -1.04047619e+01,
          3.00714286e+01,   1.05476190e+01,   3.00714286e+01,
          3.00714286e+01,   6.50000000e+00,   8.24523810e+01,
         -6.42857143e-01,   9.11904762e+00,  -6.42857143e-01,
         -6.42857143e-01,  -2.42142857e+01,  -8.97619048e+00,
         -2.85000000e+01,  -8.97619048e+00,  -8.97619048e+00,
         -3.25476190e+01,   1.19761905e+01,   2.21428571e+00,
          2.21428571e+00,  -2.13571429e+01,   1.19761905e+01,
          1.19761905e+01,  -1.15952381e+01,   2.21428571e+00,
         -2.13571429e+01,  -2.13571429e+01,  -3.88571429e+01,
         -2.76666667e+01,  -1.79047619e+01,  -2.76666667e+01,
         -2.76666667e+01,  -1.58809524e+01,   1.82857143e+01,
         -1.23809524e+00,   1.82857143e+01,   1.82857143e+01,
          3.00714286e+01,   7.80952381e+00,  -1.95238095e+00,
         -1.95238095e+00,   9.83333333e+00,   7.80952381e+00,
          7.80952381e+00,   1.95952381e+01,  -1.95238095e+00,
          9.83333333e+00,   9.83333333e+00,  -1.24285714e+01,
         -2.66666667e+00,  -1.24285714e+01,  -1.24285714e+01,
         -6.42857143e-01,   8.52380952e+00,   1.63333333e+01,
          1.63333333e+01,   2.81190476e+01,   8.52380952e+00,
          8.52380952e+00,   2.03095238e+01,   1.63333333e+01,
          2.81190476e+01,   2.81190476e+01,   1.90476190e-01,
         -9.57142857e+00,  -9.57142857e+00,   2.21428571e+00,
          1.90476190e-01,   1.90476190e-01,   1.19761905e+01,
         -9.57142857e+00,   2.21428571e+00,   2.21428571e+00,
         -2.00476190e+01,  -2.00476190e+01,  -8.26190476e+00,
         -1.22380952e+01,  -4.52380952e-01,  -4.52380952e-01,
         -2.00476190e+01,  -8.26190476e+00,  -8.26190476e+00,
         -4.52380952e-01,  -2.21904762e+01,   1.82857143e+01,
         -1.23809524e+00,   1.82857143e+01,   1.82857143e+01,
          3.00714286e+01,  -1.93333333e+01,   4.90000000e+01,
         -1.93333333e+01,  -1.93333333e+01,  -7.54761905e+00,
         -2.98095238e+01,  -1.02857143e+01,  -1.02857143e+01,
          1.50000000e+00,  -2.98095238e+01,  -2.98095238e+01,
         -1.80238095e+01,  -1.02857143e+01,   1.50000000e+00,
          1.50000000e+00,  -2.07619048e+01,  -4.02857143e+01,
         -2.07619048e+01,  -2.07619048e+01,  -8.97619048e+00,
          1.90476190e-01,  -9.57142857e+00,  -9.57142857e+00,
          2.21428571e+00,   1.90476190e-01,   1.90476190e-01,
          1.19761905e+01,  -9.57142857e+00,   2.21428571e+00,
          2.21428571e+00,  -4.85714286e+00,   1.46666667e+01,
          1.46666667e+01,   2.64523810e+01,  -4.85714286e+00,
         -4.85714286e+00,   6.92857143e+00,   1.46666667e+01,
          2.64523810e+01,   2.64523810e+01,   4.19047619e+00,
          4.19047619e+00,   1.59761905e+01,  -5.57142857e+00,
          6.21428571e+00,   6.21428571e+00,   4.19047619e+00,
          1.59761905e+01,   1.59761905e+01,   6.21428571e+00,
          2.77142857e+01,   8.19047619e+00,   2.77142857e+01,
          2.77142857e+01,   1.82857143e+01,   1.72380952e+01,
          7.47619048e+00,   7.47619048e+00,  -1.95238095e+00,
          1.72380952e+01,   1.72380952e+01,   7.80952381e+00,
          7.47619048e+00,  -1.95238095e+00,  -1.95238095e+00,
         -2.03809524e+01,  -8.57142857e-01,  -8.57142857e-01,
         -1.02857143e+01,  -2.03809524e+01,  -2.03809524e+01,
         -2.98095238e+01,  -8.57142857e-01,  -1.02857143e+01,
         -1.02857143e+01,   7.52380952e+00,   7.52380952e+00,
         -1.90476190e+00,  -2.23809524e+00,  -1.16666667e+01,
         -1.16666667e+01,   7.52380952e+00,  -1.90476190e+00,
         -1.90476190e+00,  -1.16666667e+01,   9.61904762e+00,
         -1.42857143e-01,  -1.42857143e-01,  -9.57142857e+00,
          9.61904762e+00,   9.61904762e+00,   1.90476190e-01,
         -1.42857143e-01,  -9.57142857e+00,  -9.57142857e+00,
         -1.06190476e+01,  -1.06190476e+01,  -2.00476190e+01,
         -2.80952381e+00,  -1.22380952e+01,  -1.22380952e+01,
         -1.06190476e+01,  -2.00476190e+01,  -2.00476190e+01,
         -1.22380952e+01,   1.36190476e+01,   1.36190476e+01,
          4.19047619e+00,   3.85714286e+00,  -5.57142857e+00,
         -5.57142857e+00,   1.36190476e+01,   4.19047619e+00,
          4.19047619e+00,  -5.57142857e+00,   1.22380952e+01,
          2.80952381e+00,   2.80952381e+00,   1.06190476e+01,
          2.80952381e+00])

In [57]:
y_pos_4 = np.arange(len(fourth_order_inter))
print(min(fourth_order_inter))

plt.bar(y_pos_4,fourth_order_inter, align='center')
plt.title('4th order effects within groupings of 5 mutations')
plt.ylim((min(fourth_order_inter)-50),(max(fourth_order_inter)+50))
plt.savefig("plots/FourthOrder_13_5.png")
#plt.show()

-70.1666667
